In [55]:
import os
import numpy as np
import json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Flatten
from sklearn.model_selection import train_test_split

In [56]:
# Get the current working directory
CURRENT_DIRECTORY = os.getcwd()
# Get the parent directory
PARENT_DIRECTORY = os.path.dirname(CURRENT_DIRECTORY)
PARENT_DIRECTORY = os.path.dirname(PARENT_DIRECTORY)

# Open the config file and load its content into a dictionary
config_file = open(PARENT_DIRECTORY + '\\config\\config.json')
CONFIG_DATA = json.load(config_file)

# Close the file after loading the data
config_file.close()

In [57]:
WINDOW_SIZE = CONFIG_DATA['WINDOW_SIZE']
NUMBER_OF_FEATURES = CONFIG_DATA["NUMBER_OF_FEATURES"]
NUM_EPOCHS = CONFIG_DATA["NUMBER_OF_EPOCHS"]
CLASSICATIONS = CONFIG_DATA['CLASSES_MOTIONSPEED']

ALL_X_TRAINING_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-X-TRAIN-CLASSIFY-MOTIONSPEED-SML-RSIDESTEPS-V0.npy'
ALL_Y_TRAINING_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-Y-TRAIN-CLASSIFY-MOTIONSPEED-SML-RSIDESTEPS-V0.npy'

ALL_X_TEST_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-X-TEST-CLASSIFY-MOTIONSPEED-SML-RSIDESTEPS-V0.npy'
ALL_Y_TEST_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-Y-TEST-CLASSIFY-MOTIONSPEED-SML-RSIDESTEPS-V0.npy'

MOTION_MODEL_SAVE_PATH =  PARENT_DIRECTORY + '\\NeuralNetwork\\models\\standard-backprop-motionspeed-sml-rsidesteps-2.h5'

In [58]:
ALL_X_TRAIN_DATA = np.load(ALL_X_TRAINING_DATA_PATH)
ALL_Y_TRAIN_DATA = np.load(ALL_Y_TRAINING_DATA_PATH)

ALL_X_TEST_DATA = np.load(ALL_X_TEST_DATA_PATH)
ALL_Y_TEST_DATA = np.load(ALL_Y_TEST_DATA_PATH)

ALL_X_TRAIN_DATA = ALL_X_TRAIN_DATA[:, 7] + ALL_X_TRAIN_DATA[:, 11]

# Keep column index 3 and concatenate it with the sum_7_and_11 array
ALL_X_TEST_DATA = ALL_X_TEST_DATA[:, 7] + ALL_X_TEST_DATA[:, 11]

ALL_X_COMBINED_DATA = np.concatenate((ALL_X_TRAIN_DATA, ALL_X_TEST_DATA))
ALL_Y_COMBINED_DATA = np.concatenate((ALL_Y_TRAIN_DATA -1 , ALL_Y_TEST_DATA -1))

ALL_X_TRAIN_DATA, ALL_X_TEST_DATA, ALL_Y_TRAIN_DATA, ALL_Y_TEST_DATA = train_test_split(ALL_X_COMBINED_DATA, ALL_Y_COMBINED_DATA, test_size=0.2, random_state=42)

print(ALL_Y_TRAIN_DATA[:100])

print(ALL_X_TRAIN_DATA.shape)
print(ALL_Y_TRAIN_DATA.shape)

print(ALL_X_TEST_DATA.shape)
print(ALL_Y_TEST_DATA.shape)

[0 0 1 1 0 0 0 1 0 1 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 1
 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 1 0 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 1 0 1 0 0
 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 1 0 0]
(32175,)
(32175,)
(8044,)
(8044,)


In [59]:
# Generate random permutation indices
random_indices_1 = np.random.permutation(len(ALL_X_TRAIN_DATA))


# Shuffle the input features and target labels using the random indices
shuffled_X_TRAIN_DATA = ALL_X_TRAIN_DATA[random_indices_1]
shuffled_Y_TRAIN_DATA = ALL_Y_TRAIN_DATA[random_indices_1]

In [60]:
# Generate random permutation indices
random_indices_2 = np.random.permutation(len(ALL_X_TEST_DATA))

# Shuffle the input features and target labels using the random indices
shuffled_X_TEST_DATA = ALL_X_TEST_DATA[random_indices_2]
shuffled_Y_TEST_DATA = ALL_Y_TEST_DATA[random_indices_2]

In [61]:
output_activation_function = 'sigmoid' if len(CLASSICATIONS) == 3 else 'softmax'
loss_algorithm = 'binary_crossentropy' if len(CLASSICATIONS) == 3 else 'sparse_categorical_crossentropy'
output_size = 1 if len(CLASSICATIONS) == 3 else len(CLASSICATIONS)

In [62]:
model2_motion = Sequential()
model2_motion.add(InputLayer((1)))
model2_motion.add(Dense(8, activation='relu'))
model2_motion.add(Dense(8, activation='relu'))
model2_motion.add(Dense(output_size, activation=output_activation_function))

model2_motion.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 8)                 16        
                                                                 
 dense_19 (Dense)            (None, 8)                 72        
                                                                 
 dense_20 (Dense)            (None, 1)                 9         
                                                                 
Total params: 97
Trainable params: 97
Non-trainable params: 0
_________________________________________________________________


In [63]:
# Compile the model
model2_motion.compile(optimizer='adam', loss=loss_algorithm, metrics=['acc'])

history2_motion = model2_motion.fit(shuffled_X_TRAIN_DATA, shuffled_Y_TRAIN_DATA, validation_data=(shuffled_X_TEST_DATA, shuffled_Y_TEST_DATA), epochs=NUM_EPOCHS)

# Save the model to a file
model2_motion.save(MOTION_MODEL_SAVE_PATH)

Epoch 1/12


1006/1006 [==============================] - 2s 1ms/step - loss: 0.6281 - acc: 0.6774 - val_loss: 0.6179 - val_acc: 0.6866
Epoch 2/12
1006/1006 [==============================] - 1s 1ms/step - loss: 0.6092 - acc: 0.6974 - val_loss: 0.6097 - val_acc: 0.6969
Epoch 3/12
1006/1006 [==============================] - 1s 1ms/step - loss: 0.6043 - acc: 0.7026 - val_loss: 0.6073 - val_acc: 0.6969
Epoch 4/12
1006/1006 [==============================] - 1s 1ms/step - loss: 0.6027 - acc: 0.7023 - val_loss: 0.6063 - val_acc: 0.6969
Epoch 5/12
1006/1006 [==============================] - 1s 1ms/step - loss: 0.6019 - acc: 0.7025 - val_loss: 0.6063 - val_acc: 0.6969
Epoch 6/12
1006/1006 [==============================] - 1s 1ms/step - loss: 0.6013 - acc: 0.7023 - val_loss: 0.6054 - val_acc: 0.6969
Epoch 7/12
1006/1006 [==============================] - 1s 1ms/step - loss: 0.6016 - acc: 0.7021 - val_loss: 0.6052 - val_acc: 0.6969
Epoch 8/12
1006/1006 [==============================] - 1s 1ms/step - los